In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [3]:
train_dir =  r'C:\Users\Maram Mohamed\Downloads\Alzheimer_s Dataset\Alzheimer_s Dataset\Alzheimer_s Dataset\Alzheimer_s Dataset\train'
test_dir =  r'C:\Users\Maram Mohamed\Downloads\Alzheimer_s Dataset\Alzheimer_s Dataset\Alzheimer_s Dataset\Alzheimer_s Dataset\test'

In [4]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [5]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [6]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 5121 images belonging to 4 classes.


In [7]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [8]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\Maram Mohamed\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

C:\Users\Maram Mohamed\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 15s 95ms/step


In [10]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 109ms/step


In [11]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.3264718	total: 162ms	remaining: 2m 41s
1:	learn: 1.2702669	total: 197ms	remaining: 1m 38s
2:	learn: 1.2252542	total: 228ms	remaining: 1m 15s
3:	learn: 1.1873070	total: 259ms	remaining: 1m 4s
4:	learn: 1.1565060	total: 300ms	remaining: 59.7s
5:	learn: 1.1272844	total: 349ms	remaining: 57.9s
6:	learn: 1.1027550	total: 411ms	remaining: 58.3s
7:	learn: 1.0816615	total: 447ms	remaining: 55.4s
8:	learn: 1.0595682	total: 487ms	remaining: 53.6s
9:	learn: 1.0410649	total: 523ms	remaining: 51.7s
10:	learn: 1.0231921	total: 556ms	remaining: 50s
11:	learn: 1.0058024	total: 592ms	remaining: 48.7s
12:	learn: 0.9920486	total: 626ms	remaining: 47.5s
13:	learn: 0.9800173	total: 661ms	remaining: 46.6s
14:	learn: 0.9686647	total: 694ms	remaining: 45.6s
15:	learn: 0.9598806	total: 724ms	remaining: 44.5s
16:	learn: 0.9488677	total: 756ms	remaining: 43.7s
17:	learn: 0.9387630	total: 788ms	remaining: 43s
18:	learn: 0.9302754	total: 820ms	remaining: 42.4s
19:	learn: 0.9215783	total: 867ms	remainin

In [12]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [13]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 58.33%


In [14]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [15]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [16]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model(r'C:\Users\Maram Mohamed\Downloads\Telegram Desktop\cnn_feature_extractor7.h5')
catboost_classifier = joblib.load(r'C:\Users\Maram Mohamed\Downloads\Telegram Desktop\catboost_classifier.pkl')  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = r'C:\Users\Maram Mohamed\Downloads\Alzheimer_s Dataset\Alzheimer_s Dataset\Alzheimer_s Dataset\test\MildDemented\30 (4).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
The image is classified as: MildDemented


C:\Users\Maram Mohamed\AppData\Local\Temp\ipykernel_25068\1681810196.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(prediction[0])
